In [40]:
from bs4 import BeautifulSoup
import os
from tqdm import tqdm

In [50]:
scrape = [] # List to store all ads (contains duplicates)
historybuffer = []

In [51]:
# Set up progress bar
pbar = tqdm(total=480) 
count = 0
# Looping through all 480 files
for filename in os.listdir("./data"):
    # Only consider .html files
    if filename.endswith(".html"):
        # Parse HTML into BeutifulSoup object
        html = BeautifulSoup(open("data/" + filename), "html.parser")
        # Get list of all <h2> tags matching the class name
        h2s = html.find_all("h2", class_ = "classified_date default_top_margin")
    
        # Website layout varies through time. Some files give an empty h2s list.
        # Therefore, I cater for both eventualities, starting with the newer layout.
        if len(h2s) == 0:
            # Get list of all <li> tags
            lis = html.find_all("li")
            for li in lis: # Append all ad text to scrape list
                adtext = li.text.replace("\n", "")
                if adtext not in historybuffer:
                    scrape.append(adtext)
                    historybuffer.append(adtext)
                    
                    
        # Catering for the older website style
        else:
            # Get list of all <ul> tags
            uls = html.find_all("ul", class_ = "classified_list")
            for ul in uls: 
                # Get list of all <li> tags
                lis = ul.find_all("li")
                for li in lis: # Append all ad text to scrape list
                    adtext = li.text.replace("\n", "")
                    if adtext not in historybuffer:
                        scrape.append(adtext)
                        historybuffer.append(adtext)
                    
    pbar.update(1)
pbar.close()

100%|██████████| 480/480 [06:05<00:00,  1.31it/s]


In [53]:
len(scrape)

89083

In [ ]:
scrapedict = {}